In [ ]:
!pip install -q tensorflow-model-optimization

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 238.9/238.9 KB 11.5 MB/s eta 0:00:00


# TensorFlow Quantization Example

## Traing TensorFlow model on MNIST dataset

In [ ]:
import tensorflow as tf

# Load the MNIST dataset
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

# Normalize the input data
x_train, x_test = x_train / 255.0, x_test / 255.0

# Flatten the input data
x_train = x_train.reshape(-1, 784)
x_test = x_test.reshape(-1, 784)

# Convert the labels to one-hot encoding
y_train = tf.keras.utils.to_categorical(y_train)
y_test = tf.keras.utils.to_categorical(y_test)

# Define the model
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(64, input_shape=(784,), activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(x_train, y_train, epochs=5)

# Evaluate the model
loss, accuracy = model.evaluate(x_test, y_test)
print('Test loss:', loss)
print('Test accuracy:', accuracy)

# Save the model
model.save('mnist_model.h5')

11490434/11490434 [==============================] - 1s 0us/step
Epoch 1/5
1875/1875 [==============================] - 7s 2ms/step - loss: 0.2812 - accuracy: 0.9195
Epoch 2/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.1254 - accuracy: 0.9628
Epoch 3/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0895 - accuracy: 0.9733
Epoch 4/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0684 - accuracy: 0.9788
Epoch 5/5
313/313 [==============================] - 1s 2ms/step - loss: 0.0889 - accuracy: 0.9739
Test loss: 0.08889547735452652
Test accuracy: 0.9739000201225281


## Apply Post-Training Quantization

In [ ]:
# Load the saved model
model = tf.keras.models.load_model('mnist_model.h5')

# Convert the model to TensorFlow Lite format
converter = tf.lite.TFLiteConverter.from_keras_model(model)

# Enable post-training quantization
converter.optimizations = [tf.lite.Optimize.OPTIMIZE_FOR_SIZE]

# Convert the model
tflite_model = converter.convert()

# Save the model
with open('mnist_model_quantized.tflite', 'wb') as f:
  f.write(tflite_model)

## Apply Quantization-Aware Training

In [ ]:
import tensorflow_model_optimization as tfmot

# q_aware stands for for quantization aware.
q_aware_model = tfmot.quantization.keras.quantize_model(model)

# `quantize_model` requires a recompile.
q_aware_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# train the quantized model
q_aware_model.fit(x_train, y_train, batch_size=500, epochs=5, validation_split=0.1)

_, baseline_model_accuracy = model.evaluate(
    x_test, y_test, verbose=0)

_, q_aware_model_accuracy = q_aware_model.evaluate(
   x_test, y_test, verbose=0)

print('Baseline test accuracy:', baseline_model_accuracy)
print('Quant test accuracy:', q_aware_model_accuracy)

# Save the quantized model
model.save('mnist_quantized_model.h5')

Epoch 1/5
108/108 [==============================] - 1s 7ms/step - loss: 0.0443 - accuracy: 0.9881 - val_loss: 0.0259 - val_accuracy: 0.9925
Epoch 2/5
108/108 [==============================] - 1s 5ms/step - loss: 0.0260 - accuracy: 0.9927 - val_loss: 0.0307 - val_accuracy: 0.9912
Epoch 3/5
108/108 [==============================] - 1s 5ms/step - loss: 0.0215 - accuracy: 0.9943 - val_loss: 0.0296 - val_accuracy: 0.9905
Epoch 4/5
108/108 [==============================] - 1s 5ms/step - loss: 0.0182 - accuracy: 0.9958 - val_loss: 0.0299 - val_accuracy: 0.9903
Epoch 5/5
108/108 [==============================] - 1s 5ms/step - loss: 0.0156 - accuracy: 0.9963 - val_loss: 0.0319 - val_accuracy: 0.9907
Baseline test accuracy: 0.9739000201225281
Quant test accuracy: 0.978600025177002


# PyTorch Quantization Example

## Training PyTorch Model on MNIST dataset

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms

# Set the device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Load the MNIST dataset
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])
trainset = torchvision.datasets.MNIST(root='.', train=True, download=True, transform=transform)
testset = torchvision.datasets.MNIST(root='.', train=False, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True, num_workers=2)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False, num_workers=2)

# Define the model
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(28 * 28, 512)
        self.fc2 = nn.Linear(512, 512)
        self.fc3 = nn.Linear(512, 10)

    def forward(self, x):
        x = x.view(-1, 28 * 28)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

model = Net().to(device)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

# Train the model
for epoch in range(5):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):


        # Get the inputs and labels
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        # Zero the gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)

        # Compute the loss
        loss = criterion(outputs, labels)

        # Backward pass
        loss.backward()

        # Optimize
        optimizer.step()

        # Print the statistics
        running_loss += loss.item()
        if i % 1000 == 999:  # print every 1000 mini-batches
            print('[%d, %5d] loss: %.3f' % (epoch + 1, i + 1, running_loss / 1000))
            running_loss = 0.0

# Evaluate the model
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (100 * correct / total))

# Save the model
torch.save(model, "mnist_model.pt")

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ./MNIST/raw/train-images-idx3-ubyte.gz to ./MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ./MNIST/raw/train-labels-idx1-ubyte.gz to ./MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ./MNIST/raw/t10k-images-idx3-ubyte.gz to ./MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ./MNIST/raw/t10k-labels-idx1-ubyte.gz to ./MNIST/raw

Accuracy of the network on the 10000 test images: 97 %


## Apply Post-Training Quantization

In [ ]:
import torch
import torch.quantization

# Load the saved model
model = torch.load('mnist_model.pt')

# Set the model to evaluation mode
model.eval()

# Convert the model to quantized version
quantized_model = torch.quantization.quantize_dynamic(
    model, {torch.nn.Linear}, dtype=torch.qint8
)

# Save the model
torch.save(quantized_model, 'mnist_model_quantized2.pt')

## Quantization-Aware Training

In [ ]:
# Enable quantization-aware training
model.qconfig = torch.quantization.default_qconfig

# Convert the model to quantized version
quantized_model = torch.quantization.convert(model, inplace=False)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(quantized_model.parameters(), lr=0.01, momentum=0.9)

# Train the model
for epoch in range(5):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # Get the inputs and labels
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        # Zero the gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = quantized_model(inputs)

        # Compute the loss
        loss = criterion(outputs, labels)

        # Backward pass
        loss.backward()

        # Optimize
        optimizer.step()

        # Print the statistics
        running_loss += loss.item()
        if i % 1000 == 999:  # print every 1000 mini-batches
            print('[%d, %5d] loss: %.3f' % (epoch + 1, i + 1, running_loss / 1000))
            running_loss = 0.0

# Evaluate the model
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = quantized_model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (100 * correct / total))

# Save the model
torch.save(quantized_model, 'mnist_model_quantized.pt')

Accuracy of the network on the 10000 test images: 98 %
